In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict, cross_validate
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import math
from sklearn.metrics import accuracy_score as acc_sam
from functools import reduce
from scipy.stats.stats import pearsonr
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, recall_score, precision_score,f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report
# from sklearn.lda import LDA
# from sklearn.qda import QDA
from scipy import linalg
import matplotlib as mpl
from numpy import *
import math
from scipy.stats import multivariate_normal
from sklearn.naive_bayes import GaussianNB
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier 
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR
import sympy as sp
from sklearn.impute import SimpleImputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
import torch #python #keras #tensorflow #pytorch
import torch.nn as nn
from torch.autograd import Variable

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from math import e,sqrt,sin,cos

import IPython
import numpy as np
import wave
import warnings
warnings.filterwarnings('ignore')
# from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from sklearn.decomposition import FastICA

from scipy.io import wavfile

import re
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
from bs4 import BeautifulSoup
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

# py.init_notebook_mode(connected=True)


import wordcloud
from wordcloud import WordCloud, STOPWORDS
import itertools
import string
import collections
from collections import Counter


import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

import tensorflow as tf

In [2]:
data = pd.read_csv("mbti_1.csv")

In [3]:
data.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


The first thing I want to do is turn the personality types into numbered categories, ranging from 0 to 15. These will serve as the output of the model when it is categorizing people.

In [4]:
types = np.unique(data.type.values)

In [5]:
def get_type_index(string):
    return list(types).index(string)

In [6]:
data['type_index'] = data['type'].apply(get_type_index)

In [7]:
data.posts.values[0]

"'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-home-design.jpg ...

Now, I want to clean the text to get rid of hyperlinks, puncuation, and anything else that's cluttering up the text. Specifically, the pipe (|) character seems like it separates different posts, but without spaces between them. First, I'm going to replace the pipes with spaces so that the tokenizer won't parse those parts as one long word. Then, I'll clean up the rest.

In [8]:
import string
import re

def clean_text(text):
    regex = re.compile('[%s]' % re.escape('|'))
    text = regex.sub(" ", text)
    words = str(text).split()
    words = [i.lower() + " " for i in words]
    words = [i for i in words if not "http" in i]
    words = " ".join(words)
    words = words.translate(words.maketrans('', '', string.punctuation))
    return words


In [9]:
data['cleaned_text'] = data['posts'].apply(clean_text)

In [11]:
data.head()

,type,posts,type_index,cleaned_text
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,8,enfp and intj moments sportscenter not t...
1,ENTP,'I'm finding the lack of me in these posts ver...,3,im finding the lack of me in these pos...
2,INTP,'Good one _____ https://www.youtube.com/wat...,11,good one of course to which i say i ...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",10,dear intp i enjoyed our conversation the...
4,ENTJ,'You're fired.|||That's another silly misconce...,2,youre fired thats another silly misconcep...


Now, we split the data into training, testing, and validation sets,

In [12]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data)
train, val = train_test_split(train)

In [16]:
one_hot_labels = tf.keras.utils.to_categorical(train.type_index.values, num_classes=16)
val_labels= tf.keras.utils.to_categorical(val.type_index.values, num_classes=16)

That one didn't work too well, either. Looks like I'm going to have to bring out the big guns. BERT is arguably the most powerful transformer out there right now. For more information: https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270

In [ ]:
!pip install transformers

In [26]:
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-large-uncased')

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [27]:
maxlen = 1500

train_input_ids = [tokenizer.encode(str(i), max_length = maxlen , pad_to_max_length = True) for i in train.cleaned_text.values]
val_input_ids = [tokenizer.encode(str(i), max_length = maxlen , pad_to_max_length = True) for i in val.cleaned_text.values]

In [28]:
def create_model(): 
    input_word_ids = tf.keras.layers.Input(shape=(maxlen,), dtype=tf.int32,
                                           name="input_word_ids")
    bert_layer = transformers.TFBertModel.from_pretrained('bert-large-uncased')
    bert_outputs = bert_layer(input_word_ids)[0]
    pred = tf.keras.layers.Dense(16, activation='softmax')(bert_outputs[:,0,:])
    
    model = tf.keras.models.Model(inputs=input_word_ids, outputs=pred)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.00001), metrics=['accuracy'])
    return model

In [29]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()
    
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 1500)]            0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 1500, 1024), (Non 335141888 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                16400     
Total params: 335,158,288
Trainable params: 335,158,288
Non-trainable params: 0
_________________________________________________________________


In [30]:
batch_size = 16

model.fit(np.array(train_input_ids), one_hot_labels,validation_data = (np.array(val_input_ids), val_labels),
          verbose = 1, epochs = 20, batch_size = batch_size,  callbacks = [tf.keras.callbacks.EarlyStopping(patience = 5)])

Epoch 1/20
305/305 [==============================] - 252s 828ms/step - loss: 2.2279 - accuracy: 0.2562 - val_loss: 1.9503 - val_accuracy: 0.4222
Epoch 2/20
305/305 [==============================] - 228s 749ms/step - loss: 1.5730 - accuracy: 0.5323 - val_loss: 1.4872 - val_accuracy: 0.5605
Epoch 3/20
305/305 [==============================] - 228s 748ms/step - loss: 1.2859 - accuracy: 0.6171 - val_loss: 1.4022 - val_accuracy: 0.5907
Epoch 4/20
305/305 [==============================] - 228s 748ms/step - loss: 1.1286 - accuracy: 0.6555 - val_loss: 1.2860 - val_accuracy: 0.6122
Epoch 5/20
305/305 [==============================] - 228s 748ms/step - loss: 1.0143 - accuracy: 0.6971 - val_loss: 1.2745 - val_accuracy: 0.6275
Epoch 6/20
305/305 [==============================] - 228s 748ms/step - loss: 0.8754 - accuracy: 0.7315 - val_loss: 1.2630 - val_accuracy: 0.6441
Epoch 7/20
305/305 [==============================] - 228s 748ms/step - loss: 0.7283 - accuracy: 0.7762 - val_loss: 1.2894 -

In [31]:
test_input_ids = [tokenizer.encode(str(i), max_length = maxlen , pad_to_max_length = True) for i in test.cleaned_text.values]
test_labels= tf.keras.utils.to_categorical(test.type_index.values, num_classes=16)

In [32]:
model.evaluate(np.array(test_input_ids), test_labels)

68/68 [==============================] - 20s 292ms/step - loss: 1.5094 - accuracy: 0.6810


[1.5094053745269775, 0.6809589862823486]